In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plot

In [2]:
df = pd.read_csv("../Data/Copy of hackathon_customerData_challenge2.csv")

# Drop rows where laod power is 0
df = df[df['load_power_W'] != 0.0]

# Convert date column from str to datetime
df['Date_UTC'] = pd.to_datetime(df['Date_UTC'])

df['Date_NZDT'] = df['Date_UTC'] + pd.Timedelta(hours=13)
df.insert(2, 'Date_NZDT', df.pop('Date_NZDT'))
df.head()

,customerID,Date_UTC,Date_NZDT,Date_NZDate,date_settlementPeriod,load_power_W,weather_Temp_C
0,793,2022-10-21 18:00:00+00:00,2022-10-22 07:00:00+00:00,2022-10-22,15,4099.0,NaN
1,793,2022-10-21 18:15:00+00:00,2022-10-22 07:15:00+00:00,2022-10-22,15,5566.0,NaN
2,793,2022-10-21 18:30:00+00:00,2022-10-22 07:30:00+00:00,2022-10-22,16,3565.5,NaN
3,793,2022-10-21 18:45:00+00:00,2022-10-22 07:45:00+00:00,2022-10-22,16,3420.5,NaN
4,793,2022-10-21 19:00:00+00:00,2022-10-22 08:00:00+00:00,2022-10-22,17,3236.5,NaN


In [4]:
def delete_rows_before_midnight(customer_id):
    # chosen_id = (df['customerID'] == customer_id) 
    # timestamps = df.loc[chosen_id, 'Date_NZDT']
    # first_midnight_occurrence = timestamps[timestamps.dt.time == pd.Timestamp('00:00:00').time()].head(1)
    # df['customerID'] = df['customerID'][first_midnight_occurrence:]

    #Duplicates the df columns for removing every row before midnight 
    result_df = pd.DataFrame(columns=df.columns)
    for customer_id, group in df.groupby('customerID'):
        midnight_index = group[group['Date_UTC'].dt.time == pd.Timestamp('00:00:00').time()].index.min()
        result_df = pd.concat([result_df, group.loc[midnight_index:]])

    return result_df
# Delete rows before the first midnight occurrence for each customer
df_filtered = delete_rows_before_midnight(df)

# Update the original DataFrame 'df' with the modified 'df_filtered'
df = df_filtered.copy()

In [5]:
# Spring = 0
# Summer = 1
# Autumn = 2
# Winter = 3

def get_season(date):
    month = date.month
    if month in [3, 4, 5]:
        return 0
    elif month in [6, 7, 8]:
        return 1
    elif month in [9, 10, 11]:
        return 2
    else:
        return 3

# Assuming 'Date_NZDT' is your date column
df['Season'] = df['Date_NZDT'].apply(get_season)
df.head()

,customerID,Date_UTC,Date_NZDT,Date_NZDate,date_settlementPeriod,load_power_W,weather_Temp_C,Season
8541642,1,2022-10-15 00:00:00+00:00,2022-10-15 13:00:00+00:00,2022-10-15,27,279.0,NaN,2
8541643,1,2022-10-15 00:15:00+00:00,2022-10-15 13:15:00+00:00,2022-10-15,27,1561.0,NaN,2
8541644,1,2022-10-15 00:30:00+00:00,2022-10-15 13:30:00+00:00,2022-10-15,28,2484.5,NaN,2
8541645,1,2022-10-15 00:45:00+00:00,2022-10-15 13:45:00+00:00,2022-10-15,28,2270.5,NaN,2
8541646,1,2022-10-15 01:00:00+00:00,2022-10-15 14:00:00+00:00,2022-10-15,29,322.5,NaN,2


In [6]:
def day(num):
    return slice((num - 1) * 96, num * 96 + 1)

day(1)

slice(0, 97, None)

In [24]:
n = df.size//2

#Defining the training set for the data
train_df       = df[:n].fillna(0)
train_df_power = train_df['load_power_W'].values
train_df_temp  = train_df[:n]['weather_Temp_C'].values
train_parse    = [train_df_power, train_df_temp]
train_df_date  = train_df[:n]['Date_NZDT'].values

#Defining the testing set for the data
test_df       = df[n:df.size].fillna(0)
test_df_power = test_df[n:df.size]['load_power_W']
test_df_temp  = test_df[n:df.size]['weather_Temp_C']
test_parse    = [test_df_power, test_df_temp]
test_df_date  = test_df[n:df.size]['Date_NZDT'].values

#Checks if the GPU is available 
if torch.cuda.is_available():
  device = torch.device('gpu')
else:
  device = torch.device('cpu')

#Creates the training and testing data as tensors
train_data = torch.tensor(train_parse)
test_data  = torch.tensor(test_parse)

train_data

tensor([[ 279.0000, 1561.0000, 2484.5000,  ...,  249.0000, 1153.0000,
          280.0000],
        [   0.0000,    0.0000,    0.0000,  ...,   16.8450,   17.4300,
           17.2800]], dtype=torch.float64)

In [22]:
class TorchModel(torch.nn.Module):
  
  def __init__(self):
    super(TorchModel, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(1000, 500),
      nn.ReLU(),
      nn.Linear(500, 250),
      nn.ReLU(),
      nn.Linear(250, 25),
    )
    
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [25]:
model = TorchModel().to(device)
print(model)

TorchModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1000, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=250, bias=True)
    (3): ReLU()
    (4): Linear(in_features=250, out_features=25, bias=True)
  )
)


In [27]:
logits = model(train_data)


RuntimeError: mat1 and mat2 must have the same dtype